In [41]:
import pandas as pd
import numpy as np
import os 
#import torch
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors  # K-Nearest Neighbors
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.tree import DecisionTreeClassifier  # Decision Tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scraping import get_ingredients
#import faiss

In [42]:
# directory = "World-Happiness-Report-Model"
# filename = "WHR2018Chapter2OnlineData.xls"

# Combining the directory and filename into a full path
file_path = r"./dataset_products/skincare_products_clean.csv"
df = pd.read_csv(file_path, header=0)

In [43]:
df

,product_name,product_url,product_type,clean_ingreds,price
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,https://www.lookfantastic.com/cerave-facial-mo...,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,https://www.lookfantastic.com/the-ordinary-hya...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20
3,AMELIORATE Transforming Body Lotion 200ml,https://www.lookfantastic.com/ameliorate-trans...,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50
4,CeraVe Moisturising Cream 454g,https://www.lookfantastic.com/cerave-moisturis...,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00
...,...,...,...,...,...
1133,Elemis Life Elixirs Embrace Bath and Shower El...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1134,Love Boo Splendidly Soothing Bath Soak (250ml),https://www.lookfantastic.com/love-boo-splendi...,Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99
1135,Elemis Life Elixirs Fortitude Bath and Shower ...,https://www.lookfantastic.com/elemis-life-elix...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1136,Connock London Kukui Oil Soothing Bath & Showe...,https://www.lookfantastic.com/connock-london-k...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00


In [44]:
df.isna().sum()

product_name     0
product_url      0
product_type     0
clean_ingreds    0
price            0
dtype: int64

In [45]:
df.drop(columns=['product_url'], axis=1, inplace=True)
df

,product_name,product_type,clean_ingreds,price
0,The Ordinary Natural Moisturising Factors + HA...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20
3,AMELIORATE Transforming Body Lotion 200ml,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50
4,CeraVe Moisturising Cream 454g,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00
...,...,...,...,...
1133,Elemis Life Elixirs Embrace Bath and Shower El...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1134,Love Boo Splendidly Soothing Bath Soak (250ml),Bath Oil,"['sodium lauroyl', 'sodium cocoamphoacetate', ...",£10.99
1135,Elemis Life Elixirs Fortitude Bath and Shower ...,Bath Oil,"['prunus amygdalus dulcis', 'tipa-laureth sulf...",£55.00
1136,Connock London Kukui Oil Soothing Bath & Showe...,Bath Oil,"['capric triglyceride', 'peg-40 sorbitan perol...",£36.00


In [46]:
# #using TDIF 
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# df['ingredients_text'] = df['clean_ingreds'].apply(''.join(df['clean_ingreds']))

# df

In [47]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Train Word2Vec on ingredients lists
ingredient_sentences = df['clean_ingreds'].tolist()  # List of lists of ingredients
w2v_model = Word2Vec(sentences=ingredient_sentences, vector_size=100, window=3, min_count=1, workers=4)


In [48]:
# Function to get the average embedding for a list of ingredients
def get_average_embedding(ingredients):
    # Get the embedding for each ingredient and calculate the mean
    embeddings = [w2v_model.wv[ingredient] for ingredient in ingredients if ingredient in w2v_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(w2v_model.vector_size)

# Apply the function to get an embedding for each product
df['product_embedding'] = df['clean_ingreds'].apply(get_average_embedding)


In [49]:
embedding_matrix = np.vstack(df['product_embedding'].values)

# Compute cosine similarity between products
similarity_matrix = cosine_similarity(embedding_matrix)

# Convert to DataFrame for easy interpretation
similarity_df = pd.DataFrame(similarity_matrix, index=df['product_name'], columns=df['product_name'])
print(similarity_df)

product_name                                        The Ordinary Natural Moisturising Factors + HA 30ml  \
product_name                                                                                              
The Ordinary Natural Moisturising Factors + HA ...                                           1.000000     
CeraVe Facial Moisturising Lotion SPF 25 52ml                                                0.995696     
The Ordinary Hyaluronic Acid 2% + B5 Hydration ...                                           0.981502     
AMELIORATE Transforming Body Lotion 200ml                                                    0.997711     
CeraVe Moisturising Cream 454g                                                               0.996344     
...                                                                                               ...     
Elemis Life Elixirs Embrace Bath and Shower Eli...                                           0.977057     
Love Boo Splendidly Soothing Bath Soa

Scraping code

In [50]:
# #with the scraping 
# def find_similar_scraping(user_product_url, top_n = 5):
#     user_ingredients = get_ingredients(user_product_url)
#     if not user_ingredients:
#         print("No ingredients found.")
#         return 
    
#     user_embedding = get_average_embedding(user_ingredients)
#     similarities = cosine_similarity([user_embedding], embedding_matrix).flatten()

#     top_index = similarities.argsort()[-top_n:][::-1]
#     similar_products = df.iloc[top_index][['product_name', 'clean_ingreds']]
#     similar_products['similarity_score'] = similarities[top_index]

#     # Display results with similarity scores
#     # print(similar_products.to_string(index=False))

#     print(f"Top {top_n} similar products to the one at '{user_product_url}':")
#     print(similar_products)

# # Example usage with a URL input
# user_url = "https://incidecoder.com/products/glossier-futuredew"
# find_similar_scraping(user_url)

In [51]:
def get_top_similar_products(product_name, top_n=5):
    if product_name not in df['product_name'].values:
        print("Product not found.")
        return

    # Get the index of the product
    product_idx = df[df['product_name'] == product_name].index[0]

    # Compute cosine similarity of the target product with all others
    similarities  = cosine_similarity([embedding_matrix[product_idx]], embedding_matrix).flatten()

    # Get top N most similar products (excluding the product itself)
    top_indices = similarities.argsort()[-top_n:][::-1]
    similar_products = df.iloc[top_indices][['product_name', 'clean_ingreds']]
    similar_products['similarity_score'] = similarities[top_indices]

    # Display results with similarity scores
    print(f"Top {top_n} similar products to '{product_name}':")
    print(similar_products.to_string(index=False))

# Example usage
get_top_similar_products("REN Clean Skincare Radiance Perfection Serum", top_n=5)

Top 5 similar products to 'REN Clean Skincare Radiance Perfection Serum':
                                     product_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           clean_ingreds  sim

In [52]:
def get_top_similar_products(product_name, top_n = 5):
    if product_name not in df['product_name'].values:
        print("Product Not Found ") #we can change this later to the produtt ingredients 

In [53]:
# #import importlib.util

# import importlib.resources
# # try:
# #     from sentence_transformers import SentenceTransformer
# # except ModuleNotFoundError:
# #     print("SentenceTransformer module could not be imported, continuing without it.")


# # # Load a pre-trained model
# # model = SentenceTransformer('all-MiniLM-L6-v2')
# try:
#     from sentence_transformers import SentenceTransformer
#     model_available = True
# except ModuleNotFoundError:
#     print("SentenceTransformer module could not be imported, skipping model loading.")
#     model_available = False

# # Use SentenceTransformer only if it was successfully imported
# if model_available:
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     # Proceed with code that requires the model
# else:
#     # Handle alternative logic if SentenceTransformer is unavailable
#     print("Model loading is skipped due to import failure.")

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('all-MiniLM-L6-v2')
# Convert text descriptions to embeddings

vectorized_data = model.encode(df)

NameError: name 'model' is not defined

In [ ]:
# models = {
#    'Logistic Regression': LogisticRegression(max_iter=200),
#    'Random Forest': RandomForestRegressor(n_estimators=100, random_state = 1),
#    # 'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state = 12),
#    knn = NearestNeighbors(n_neighbors=5, metric='cosine')

#    'D Tree': DecisionTreeClassifier()
# }

knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(vectorized_data)


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

In [ ]:
text = "hydrating hand cream"
text_embedding = model.encode([text])

distances, indices = knn.kneighbors(text_embedding)

print("Indicies of similar products:", indices )
print("Distances of similar products:", distances )
print("Similar products: \n", [df.iloc[i] for i in indices[0]])




In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(vectorized_data)

plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=df['cluster'], cmap='viridis')
plt.colorbar()
plt.title("Product Clusters")
plt.show()
